This notebook is for preprocessing the data and getting it ready to be clustered. This is done by exploring the data, getting rid of outliers, 
and converting data to binary to only check for presence of a feature. I chose to use a variance threshhold for some feature reduction by only 
getting rid of features where all values were identical. If a feature is just the exact same (present or not present) for every sample of data, it 
contriubutes nothing but noise. I chose not to use Principal Componenet Analysis (PCA) because it assumes the data is continuouse, not binary, causing 
potential issues due to assumptions. Multiple Correspondence Analysis (MCA) would also work for dimensionality reduction, but would likely cause a loss 
in information.

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
import re

In [3]:
df = pd.read_csv("../data/parquets/raw_parquet.csv")

/var/folders/k_/qdqmhtsx7mz24yl5hmlfnjz80000gn/T/ipykernel_22593/1490154515.py:1: DtypeWarning: Columns (0,3,4,6,7,8,12,13,15,16,19,22,23,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,46,47,48,49,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,82,84,85,86,87,88,89,90,91,92,94,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,124,125,126,127,128,129,130,131,132,133,134,135,136,138,139,140,144,145,146,147,148,149,150,151,152,153,154,155,156,159,165,168,169,170,173,174,176,177,178,179,180,181,182,183,184,185,186,187,188,189,191,192,193,194,197,199,200,201,221,222,223,224,225,228,229,230,231,232,233,234,235,236,237,242,243,244,245,246,247,248,249,250,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,307,310,312,313,314,315,316,319,320,321,322,323,324,325

In [14]:
#use this to see value counts of each column. The builtin value_counts function in the pandas libary is hard to read, so with val_count, it makes it a bit easier

def val_count(data):
    counter = 0
    for i in data.columns:
        print(counter)
        print(data[i].value_counts())
        counter += 1
        print("=========================="*5)
    print(data.shape)

In [ ]:
val_count(df)

Some features may be unnecessary in the in analysis, especially features that pertain more to the use of the app and its features like line and circle colors for example. It is also a very necessary step to try and reduce the cardinality of the data when possible so as to reduce the noise later on and improve processing time.

In [18]:
columns_to_drop = []
for col in df.columns:
    if re.search("^properties_symbology", col):
        columns_to_drop.append(col)
    if re.search("_id$", col):
        columns_to_drop.append(col)
    if re.search("_timestamp$", col):
        columns_to_drop.append(col)

df = df.drop(columns= columns_to_drop)

print("Columns Dropped:")
for col in columns_to_drop:
    print(f"\t{col}")



Columns Dropped:
	properties_viewed_timestamp
	properties_modified_timestamp
	properties_symbology_circleColor
	properties_id
	properties_symbology_lineColor
	properties_symbology_lineWidth
	properties_symbology_lineDasharray
	properties_sed_strat_section_strat_section_id
	properties_strat_section_id
	properties_symbology_fillColor
	properties_orientation_id
	properties_custom_fields_osm_id
	properties_orientation_modified_timestamp
	properties_custom_fields_id
	properties_orientation_unix_timestamp


In [20]:
#getting rid of duplicates

df.drop_duplicates(inplace = True)
df.shape

(1359415, 465)

In [21]:
def binary_simplification(df):
    """converts a pandas dataframe into binary based off of presence in data

    Args:
        df (pandas.DataFrame): a pandas dataframe of the data that needs to be converted to binary

    Returns:
        pandas.DataFrame: a new dataframe that has now been converted to binary
    """
    df_new = df.copy()
    binary_col_data = {}
    columns_to_drop = []
    
    for col in df.columns:
        # print(f"Converting {col} to binary")
        binary_col_data[col] = df_new[col].replace('', np.nan).notna().astype(int)
        columns_to_drop.append(col)
            
    df_new = df_new.drop(columns=columns_to_drop)
    
    # Add all new binary columns in one go using pd.concat
    if binary_col_data:
        df_new = pd.concat([df_new, pd.DataFrame(binary_col_data, index=df_new.index)], axis=1)
        
    return df_new

In [23]:
df = binary_simplification(df)
# val_count(df)

In [24]:
# uses variance threshold for feature reduction, removes features where all values are identical

selector = VarianceThreshold(threshold=0) #removes all features with low variance in 100% of samples  (you do (1 - percentage of same values) * percentage of vaues that are the same)
selector.fit_transform(df)

cols_idxs = selector.get_support(indices=True)
df = df.iloc[:,cols_idxs]

print(df.shape)
# val_count(df)


(1359415, 442)


In [25]:
df.to_csv("../data/parquets/processed_parquet.csv", index = False)
